In [42]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 16, 8

Populating the interactive namespace from numpy and matplotlib


In [43]:
tr = pd.read_csv('input/train.csv',
                 usecols=['Id',
                          'minutes_past',
                          'radardist_km',
                          'Ref',
                          'Ref_5x5_50th',
                          'RefComposite',
                          'RefComposite_5x5_50th',
                          'Expected'])

In [44]:
te = pd.read_csv('input/test.csv',
                 usecols=['Id',
                          'minutes_past',
                          'radardist_km',
                          'Ref',
                          'Ref_5x5_50th',
                          'RefComposite',
                          'RefComposite_5x5_50th'])
te['Expected'] = -1
te['Id'] += 10000000

In [45]:
d = pd.concat([tr, te])
d.index = pd.Index(range(len(d)))
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21787957 entries, 0 to 21787956
Data columns (total 8 columns):
Id                       int64
minutes_past             int64
radardist_km             float64
Ref                      float64
Ref_5x5_50th             float64
RefComposite             float64
RefComposite_5x5_50th    float64
Expected                 float64
dtypes: float64(6), int64(2)
memory usage: 1.5 GB


In [46]:
del tr
del te

In [47]:
grouped = d.groupby('Id')

In [48]:
del d

In [49]:
def compute(g, c):
    valid_time = np.zeros_like(g.minutes_past)
    valid_time[0] = g.minutes_past.iloc[0]
    for n in xrange(1, len(g.minutes_past)):
        valid_time[n] = g.minutes_past.iloc[n] - g.minutes_past.iloc[n-1]
    valid_time[-1] = valid_time[-1] + 60 - np.sum(valid_time)
    valid_time = valid_time / 60.0
    
    sum_mp = 0
    sum_ecs = 0
    sum_wcs = 0
    sum_w8c = 0
    sum_rt = 0
    for dbz, hours in zip(g[c], valid_time):
        if np.isfinite(dbz):
            mp_perhr = pow(pow(10, dbz/10)/200, 0.625)
            sum_mp = sum_mp + mp_perhr * hours
            ecs_perhr = pow(pow(10, dbz/10)/130, 0.5)
            sum_ecs = sum_ecs + ecs_perhr * hours
            wcs_perhr = pow(pow(10, dbz/10)/75, 0.5)
            sum_wcs = sum_wcs + wcs_perhr * hours
            w8c_perhr = pow(pow(10, dbz/10)/300, 0.714)
            sum_w8c = sum_w8c + w8c_perhr * hours
            rt_perhr = pow(pow(10, dbz/10)/250, 0.833)
            sum_rt = sum_rt + rt_perhr * hours
    return (sum_mp, sum_ecs, sum_wcs, sum_w8c, sum_rt)

In [50]:
def aggregate_groups(group):
    group = group.sort('minutes_past', ascending=True)
    null_ratio = (1.0 * group.isnull().sum().sum()) / (len(group) * (len(group.columns) - 2))
    vals = [
        group.iloc[0]['Id'],
        group['Expected'].mean(),
        len(group),
        null_ratio,
        group['radardist_km'].mean(),
    ]
    cols = [
        'Id',
        'Expected',
        'cnt',
        'null_ratio',
        'radardist_km'
    ]
    radar_cols = ['Ref', 'Ref_5x5_50th', 'RefComposite', 'RefComposite_5x5_50th']
    for c in radar_cols:
        mp, ecs, wcs, w8c, rt = compute(group, c)
        vals.append(mp)
        cols.append(c + '_mp')
        vals.append(ecs)
        cols.append(c + '_ecs')
        vals.append(wcs)
        cols.append(c + '_wcs')
        vals.append(w8c)
        cols.append(c + '_w8c')
        vals.append(rt)
        cols.append(c + '_rt')
    if group['Id'].mean() % 100000 == 0:
        print group['Id'].mean()
    return Series(vals, index=cols)

In [51]:
g = grouped.apply(aggregate_groups)

100000.0
200000.0
300000.0
400000.0
500000.0
600000.0
700000.0
800000.0
900000.0
1000000.0
1100000.0
10100000.0
10200000.0
10300000.0
10400000.0
10500000.0
10600000.0
10700000.0


In [52]:
del grouped

In [53]:
g.Id = g.Id.astype(int)
g.cnt = g.cnt.astype(int)

In [54]:
g.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1898570 entries, 1 to 10717625
Data columns (total 25 columns):
Id                           int64
Expected                     float64
cnt                          int64
null_ratio                   float64
radardist_km                 float64
Ref_mp                       float64
Ref_ecs                      float64
Ref_wcs                      float64
Ref_w8c                      float64
Ref_rt                       float64
Ref_5x5_50th_mp              float64
Ref_5x5_50th_ecs             float64
Ref_5x5_50th_wcs             float64
Ref_5x5_50th_w8c             float64
Ref_5x5_50th_rt              float64
RefComposite_mp              float64
RefComposite_ecs             float64
RefComposite_wcs             float64
RefComposite_w8c             float64
RefComposite_rt              float64
RefComposite_5x5_50th_mp     float64
RefComposite_5x5_50th_ecs    float64
RefComposite_5x5_50th_wcs    float64
RefComposite_5x5_50th_w8c    float64
Re

In [55]:
g[:30]
g.to_csv('tmp/grouped_mp_02.csv')